DETECTION OF YOGA POSES USING OPENPOSE AND CNN

In [ ]:
# Importing Necessary Libraries
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import cv2
import re
import numpy as np
#!pip install keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageFilter

In [ ]:
# Mounting Dataset from google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Snippet of code for visualizing effect of filters
path='/content/drive/MyDrive/yoga_dataset/'
for i in os.scandir(path):
  ct = 0
  pose = os.path.basename(i)
  path2 = os.path.join(path,pose)
  l =[]
  for j in os.listdir(path2):
    if(ct==3):
      break
    path3 = os.path.join(path2,j)
    img = cv2.imread(path3)[...,::-1]
    blur = cv2.GaussianBlur(img,(5,5),0)
    plt.subplot(151),plt.imshow(img),plt.title('Original')
    plt.xticks([]), plt.yticks([])
    plt.subplot(152),plt.imshow(blur),plt.title('Gaussian')
    plt.xticks([]), plt.yticks([])
    img_median = cv2.medianBlur(blur,5) 
    plt.subplot(153),plt.imshow(img_median),plt.title('Median filter')
    plt.xticks([]), plt.yticks([])
    img2 = Image.fromarray(img_median, 'RGB')
    im2 = img2.filter(ImageFilter.UnsharpMask(percent=150))
    plt.subplot(154),plt.imshow(im2),plt.title('Sharpened')
    plt.xticks([]), plt.yticks([])
    im3 = im2.filter(ImageFilter.EDGE_ENHANCE_MORE)
    plt.subplot(155),plt.imshow(im2),plt.title('Edge-enhanced')
    plt.xticks([]), plt.yticks([])
    plt.show()
    ct=ct+1

Without **OpenPose** and **Filters**

In [ ]:
# Code to read the images without passing through any filters
labels = []
images = []
asanas_name = []
images_path = []
images_pixels = []
mapping={}
i=0
dataset_path = '/content/drive/MyDrive/yoga_dataset/'

for directory in os.listdir(dataset_path):
    asanas_name.append(directory)
    print(directory)
    c=0
    for img in os.listdir(os.path.join(dataset_path,directory)):  
        if len(re.findall('.png',img.lower())) != 0 or len(re.findall('.jpg',img.lower())) != 0 or len(re.findall('.jpeg',img.lower())) != 0:
            img_path = os.path.join(os.path.join(dataset_path,directory),img)
            images.append(img)
            images_path.append(img_path)
            #print(img)
            img_pix = cv2.imread(img_path,1)
            images_pixels.append(cv2.resize(img_pix, (100,100)))
            labels.append(i)
            c+=1
    mapping[i]=directory
    i = i+1
    if i==20:
      break

    
print("Total labels: ", len(labels))
print("Total images: ", len(images))
print("Total images path: ", len(images_path))
print("Total asanas: ", len(asanas_name))
print("Total images_pixels: ", len(images_pixels))

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(images_pixels[0])

In [ ]:
!git clone https://github.com/misbah4064/human-pose-estimation-opencv.git
%cd human-pose-estimation-opencv/

In [ ]:
# OpenPose Code
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

width = 368
height = 368
inWidth = width
inHeight = height

net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
thr = 0.2

# Defining function which uses OpenPose
def poseDetector(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()

    return frame

In [ ]:
# Calling the function related OpenPose
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = images_path
# fimg and flabel will contain the images with OpenPose marking and corresponding labels
fimg=[]
flabel=[]
for i, img_path in enumerate(next_pix):
    img = cv.imread(img_path)
    #print(type(img))
    output = poseDetector(img)
    #cv2_imshow(output)
    fimg.append(cv2.resize(output, (100,100)))
    flabel.append(labels[i])

#plt.show()

In [ ]:
cv2_imshow(fimg[0])

In [ ]:
input = cv.imread("image.jpg")
output = poseDetector(input)
#cv2_imshow(output)

In [ ]:
import matplotlib.pyplot as plt
dgen = list(zip(images_pixels,labels))
random.shuffle(dgen)

trset,lab = zip(*dgen)



In [ ]:
labels[0]

In [ ]:
x = np.array(trset) / 255
y =  to_categorical(lab, num_classes = 20)
# Splitting the records into Training and Testing Set
xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size = 0.2, random_state=101)

print("Number of training set records ", len(xtrain))
print("Number of testing set records : ", len(xtest))

In [ ]:
imgdata = ImageDataGenerator(horizontal_flip=False,vertical_flip=False,rotation_range=0,
                             zoom_range=0.2,width_shift_range=0,height_shift_range=0,shear_range=0,fill_mode="nearest")

In [ ]:
# Importing Pre trained DenseNet121 CNN Model
densemod = tf.keras.applications.DenseNet121(input_shape=(100,100,3),include_top=False,weights='imagenet',pooling='avg')

densemod.trainable = False

In [ ]:
# Adding input,output and drop layers
ip = densemod.input
op=densemod.output

dropout_1 = tf.keras.layers.Dropout(0.25)(op)
layer_x_1 = tf.keras.layers.Dense(512, activation='relu')(dropout_1)
layer_x_2 = tf.keras.layers.Dense(128, activation='relu')(layer_x_1)
dropout_2 = tf.keras.layers.Dropout(0.20)(layer_x_2)
res = tf.keras.layers.Dense(20, activation='softmax')(dropout_2)


train_mod = tf.keras.Model(inputs=ip, outputs=res)

In [ ]:
# Training the model for 20 epochs
adam_res = tf.keras.optimizers.Adam(learning_rate=0.001)

train_mod.compile(optimizer=adam_res,loss='categorical_crossentropy',metrics=['acc'])
fresult = train_mod.fit(imgdata.flow(xtrain,ytrain,batch_size=32),validation_data=(xtest,ytest),epochs=20)

In [ ]:
# Plotting the accuracy obtained across each epoch
trainacc = fresult.history['acc']
testacc = fresult.history['val_acc']
trainloss = fresult.history['loss']
testloss = fresult.history['val_loss']
ep = range(len(trainacc))

plt.plot(ep, trainacc, 'r', label='Training accuracy')
plt.plot(ep, testacc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
pred=train_mod.predict(xtrain)

With **Openpose** but without **filters**

In [ ]:
# Code to train the model with original images passed through OpenPose library but not through filters
import matplotlib.pyplot as plt
dgen = list(zip(fimg,flabel))
random.shuffle(dgen)

tr_set,lab = zip(*dgen)

x = np.array(tr_set) / 255
y =  to_categorical(lab, num_classes = 20)

xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size = 0.2, random_state=101)

print("Number of training set records ", len(xtrain))
print("Number of testing set records : ", len(xtest))

imgdata = ImageDataGenerator(horizontal_flip=False,vertical_flip=False,rotation_range=0,
                             zoom_range=0.2,width_shift_range=0,height_shift_range=0,shear_range=0,fill_mode="nearest")

densemod = tf.keras.applications.DenseNet121(input_shape=(100,100,3),include_top=False,weights='imagenet',pooling='avg')

densemod.trainable = False
ip = densemod.input
op=densemod.output

dropout_1 = tf.keras.layers.Dropout(0.25)(op)
layer_x_1 = tf.keras.layers.Dense(512, activation='relu')(dropout_1)
layer_x_2 = tf.keras.layers.Dense(128, activation='relu')(layer_x_1)
dropout_2 = tf.keras.layers.Dropout(0.20)(layer_x_2)
res = tf.keras.layers.Dense(20, activation='softmax')(dropout_2)


train_mod = tf.keras.Model(inputs=ip, outputs=res)

adam_res = tf.keras.optimizers.Adam(learning_rate=0.001)

train_mod.compile(optimizer=adam_res,loss='categorical_crossentropy',metrics=['acc'])
fresult = train_mod.fit(imgdata.flow(xtrain,ytrain,batch_size=32),validation_data=(xtest,ytest),epochs=20)

trainacc = fresult.history['acc']
trainloss = fresult.history['loss']
testacc = fresult.history['val_acc']
testloss = fresult.history['val_loss']
ep = range(len(trainacc))

plt.plot(ep, trainacc, 'r', label='Training accuracy')
plt.plot(ep, testacc, 'b', label='Testing accuracy')
plt.title('Accuracy Analysis : training vs testing')
plt.legend(loc=0)
plt.figure()


plt.show()
print("Max accuracy:"+str(max(testacc)))

Without **OpenPose** but with **Filters**

In [ ]:
# Code to train the model with images passed through filters but not through OpenPose
labels = []
images = []
asanas_name = []
images_path = []
images_pixels = []

i=0
dataset_path = '/content/drive/MyDrive/yoga_dataset/'

for directory in os.listdir(dataset_path):
    asanas_name.append(directory)
    print(directory)
    c=0
    for img in os.listdir(os.path.join(dataset_path,directory)):  
        if len(re.findall('.png',img.lower())) != 0 or len(re.findall('.jpg',img.lower())) != 0 or len(re.findall('.jpeg',img.lower())) != 0:
            img_path = os.path.join(os.path.join(dataset_path,directory),img)
            images.append(img)
            images_path.append(img_path)
            #print(img)
            img_pix = cv2.imread(img_path,1)
            # Preprocessing
            blur = cv2.GaussianBlur(img_pix,(5,5),0)
            img_m = cv2.medianBlur(blur,5)
            img2 = Image.fromarray(img_m, 'RGB')
            im2 = img2.filter(ImageFilter.UnsharpMask(percent=150))
            im3 = im2.filter(ImageFilter.EDGE_ENHANCE_MORE)
            #appending
            images_pixels.append(cv2.resize(img_m, (100,100)))
            labels.append(i)
            c+=1
    i = i+1
    if i==20:
      break    
print("Total labels: ", len(labels))
print("Total images: ", len(images))
print("Total images path: ", len(images_path))
print("Total asanas: ", len(asanas_name))
print("Total images_pixels: ", len(images_pixels))

In [ ]:

import matplotlib.pyplot as plt
dgen = list(zip(images_pixels,labels))
random.shuffle(dgen)

tr_set,lab = zip(*dgen)

x = np.array(tr_set) / 255
y =  to_categorical(lab, num_classes = 20)

xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size = 0.2, random_state=101)

print("Number of training set records ", len(xtrain))
print("Number of testing set records : ", len(xtest))

imgdata = ImageDataGenerator(horizontal_flip=False,vertical_flip=False,rotation_range=0,
                             zoom_range=0.2,width_shift_range=0,height_shift_range=0,shear_range=0,fill_mode="nearest")

densemod = tf.keras.applications.DenseNet121(input_shape=(100,100,3),include_top=False,weights='imagenet',pooling='avg')

densemod.trainable = False
ip = densemod.input
op=densemod.output

dropout_1 = tf.keras.layers.Dropout(0.25)(op)
layer_x_1 = tf.keras.layers.Dense(512, activation='relu')(dropout_1)
layer_x_2 = tf.keras.layers.Dense(128, activation='relu')(layer_x_1)
dropout_2 = tf.keras.layers.Dropout(0.20)(layer_x_2)
res = tf.keras.layers.Dense(20, activation='softmax')(dropout_2)


train_mod = tf.keras.Model(inputs=ip, outputs=res)

adam_res = tf.keras.optimizers.Adam(learning_rate=0.001)

train_mod.compile(optimizer=adam_res,loss='categorical_crossentropy',metrics=['acc'])
fresult = train_mod.fit(imgdata.flow(xtrain,ytrain,batch_size=32),validation_data=(xtest,ytest),epochs=20)

trainacc = fresult.history['acc']
trainloss = fresult.history['loss']
testacc = fresult.history['val_acc']
testloss = fresult.history['val_loss']
ep = range(len(trainacc))

plt.plot(ep, trainacc, 'r', label='Training accuracy')
plt.plot(ep, testacc, 'b', label='Testing accuracy')
plt.title('Accuracy Analysis : training vs testing')
plt.legend(loc=0)
plt.figure()


plt.show()
print("Max accuracy:"+str(max(testacc)))

With **OpenPose and filters**

In [ ]:
#@title Default title text
# Code to train the model with original images passed through OpenPose and through filters
import matplotlib.pyplot as plt
dgen = list(zip(fimg,flabel))
random.shuffle(dgen)

tr_set,lab = zip(*dgen)

x = np.array(tr_set) / 255
y =  to_categorical(lab, num_classes = 20)

xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size = 0.2, random_state=101)

print("Number of training set records ", len(xtrain))
print("Number of testing set records : ", len(xtest))

imgdata = ImageDataGenerator(horizontal_flip=False,vertical_flip=False,rotation_range=0,
                             zoom_range=0.2,width_shift_range=0,height_shift_range=0,shear_range=0,fill_mode="nearest")

densemod = tf.keras.applications.DenseNet121(input_shape=(100,100,3),include_top=False,weights='imagenet',pooling='avg')

densemod.trainable = False
ip = densemod.input
op=densemod.output

dropout_1 = tf.keras.layers.Dropout(0.25)(op)
layer_x_1 = tf.keras.layers.Dense(512, activation='relu')(dropout_1)
layer_x_2 = tf.keras.layers.Dense(128, activation='relu')(layer_x_1)
dropout_2 = tf.keras.layers.Dropout(0.20)(layer_x_2)
res = tf.keras.layers.Dense(20, activation='softmax')(dropout_2)


train_mod = tf.keras.Model(inputs=ip, outputs=res)

adam_res = tf.keras.optimizers.Adam(learning_rate=0.001)

train_mod.compile(optimizer=adam_res,loss='categorical_crossentropy',metrics=['acc'])
fresult = train_mod.fit(imgdata.flow(xtrain,ytrain,batch_size=32),validation_data=(xtest,ytest),epochs=20)

trainacc = fresult.history['acc']
trainloss = fresult.history['loss']
testacc = fresult.history['val_acc']
testloss = fresult.history['val_loss']
ep = range(len(trainacc))

plt.plot(ep, trainacc, 'r', label='Training accuracy')
plt.plot(ep, testacc, 'b', label='Testing accuracy')
plt.title('Accuracy Analysis : training vs testing')
plt.legend(loc=0)
plt.figure()


plt.show()
print("Max accuracy:"+str(max(testacc)))

In [ ]:
pred=train_mod.predict(xtrain)

In [ ]:
cv2_imshow(xtrain[0]*255)

In [ ]:
lab=list(pred[0]).index(max(pred[0]))
print(lab)
print(mapping[lab])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
acc=[82.683,84.84,78.35,81.385]
lab=['Without OpenPose and filters','With OpenPose and Without Filters',' Without OpenPose and with filters','With OpenPose and Filters']
plt.figure(figsize=(22,12))
clrs = ['orange' if (x < max(acc)) else 'red' for x in acc]
plt.bar(range(len(acc)),acc, align='center',color=clrs,edgecolor="yellow",linewidth=5)
plt.xticks(range(len(acc)), lab,fontweight='bold', fontsize='15', horizontalalignment='center')
plt.ylabel("Accuracy",fontweight='bold', fontsize='15', horizontalalignment='center')
plt.yticks(fontweight='bold', fontsize='15')
plt.show()